In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
# It's OK if this gives a warning about using the default file formatting--this happens if you didn't open and re-save the Excel file before importing to Python.
preferences = pd.read_excel("Spring+2026+Course+Preference+Form_July+28,+2025_15.16.xlsx") # Be sure to "export labels" when exporting. #Spring+2025+Course+Preference+Form_July+25,+2024_15.42.xlsx
output_file_name = 'Spring2026_updatingParseQualtrics.xlsx' #Spring2025_test.xlsx

num_test_rows = 0 # how many test rows I created when debugging the Qualtrics form

meeting_patterns = pd.read_excel("Meeting_Patterns_July_2025.xlsx", sheet_name="Time_Options")

# Commented this out; doing the merge in the scheduling code instead.
# # Data on goal_nums, credits per prof, class credits, and cannot_teach
# # that doesn't come from Qualtrics, but that will be included with the
# # survey data in the input file (e.g., Fall_2025_v4.xlsx)
# to_merge = "Fall_2025_to_merge_with_Qualtrics.xlsx"
# Goal_nums = pd.read_excel(to_merge, sheet_name="Goal_nums")
# Credits_per_prof = pd.read_excel(to_merge, sheet_name="Credits_per_prof")
# class_credits = pd.read_excel(to_merge, sheet_name="class_credits")
# Goal_nums = pd.read_excel(to_merge, sheet_name="Goal_nums")

In [3]:
#preferences

In [4]:
# Strip the first "data" row that contains extra info about the questions
detailed_header = preferences.iloc[0]
preferences = preferences.iloc[1+num_test_rows:].copy()
#preferences

In [5]:
#preferences["FullName"] = f'{preferences["Q1"]}{preferences["Q2"]}' # This way includes some extra junk
preferences["Prof_name"] = preferences["Q1"] + preferences["Q2"]
preferences["Prof_name"]

1               AbraBrisbin
2     JulianAntolinCamarena
3              ChrisAhrendt
4              ColleenDuffy
5              MohammadAziz
6             AllisonBeemer
7               AihamHassan
8                ChloeLewis
9              AbhinavChand
10           VickiWhitledge
11               RobertBerg
12              CarolynOtto
13               SamScholze
14          KatrinaRothrock
15              KrisPresler
16              HerschelDay
17            MelissaTroudt
18            JessicaKraker
19               aBaMbirika
20                ChrisHlas
21         ChristopherDavis
22     Marie-ClaireKoissi-K
23     RyanAndJennyHarrison
24             KatieElliott
Name: Prof_name, dtype: object

## Course preferences

In [6]:
# Identify the columns and what they should be called

# Example of how to grab columns of `preferences` where the detailed_header contains certain text
#a = np.where(detailed_header == "First Name")
#detailed_header.iloc[a]
#preferences.iloc[:, a[0][0]]

# This works to avoid regular expressions, but then you have to filter out the questions that 
# say "please describe your preferences for number of preps not counting Math 10/20"
#[col for col in range(detailed_header.size) if detailed_header.iloc[col].find("Math ") > -1]

math_cols = [] # column numbers that refer to preferences about math courses
ds_cols = [] # column numbers that refer to preferences about DS courses
math_courses = [] # names of math courses, in the order that corresponds to math_cols
ds_courses = []
es_cols = [] # column numbers that refer to preferences about ES courses
es_courses = []

for col in range(detailed_header.size):
    # the [ \(a-z*\-*a-z*\)]* part is to include (in-person) and (online)
    found_math = re.findall(" - (Math [0-9]+[ \(a-z*\-*a-z*\)]*)", detailed_header.iloc[col])
    if found_math:
        math_courses.extend(found_math)
        math_cols.append(col)
    found_ds = re.findall(" - (DS [0-9]+)", detailed_header.iloc[col])
    if found_ds:
        ds_courses.extend(found_ds)
        ds_cols.append(col)
    found_es = re.findall(" - (ES [0-9]+)", detailed_header.iloc[col])
    if found_es:
        es_courses.extend(found_es)
        es_cols.append(col)
    
math_courses
#ds_courses


['Math 201',
 'Math 202',
 'Math 302',
 'Math 304',
 'Math 322',
 'Math 365',
 'Math 451',
 'Math 260',
 'Math 350',
 'Math 460',
 'Math 246 (in-person)',
 'Math 246 (online)',
 'Math 347',
 'Math 441',
 'Math 442',
 'Math 447',
 'Math 312',
 'Math 314',
 'Math 316',
 'Math 317',
 'Math 318',
 'Math 324',
 'Math 330',
 'Math 354',
 'Math 425',
 'Math 426',
 'Math 440',
 'Math 462',
 'Math 480',
 'Math 114',
 'Math 215',
 'Math 216',
 'Math 109',
 'Math 111',
 'Math 112',
 'Math 113',
 'Math 104',
 'Math 106',
 'Math 107',
 'Math 10',
 'Math 20',
 'Math 50',
 'Math 291',
 'Math 297']

In [7]:
# Extract course preference columns
columns_to_keep = [preferences.columns.get_loc("Prof_name")]
columns_to_keep.extend(math_cols)
columns_to_keep.extend(ds_cols)
columns_to_keep

course_preferences = preferences.iloc[:, columns_to_keep]
# The following line tells Pandas to infer that the columns contain floats.
# After this, filling NA's with 0 doesn't cause the column type to change,
# so it avoids the FutureWarning.
course_preferences = course_preferences.infer_objects()
#print(course_preferences.dtypes)
# Replace NA's with 0
course_preferences = course_preferences.fillna(0)
#course_preferences

In [8]:
# Change column names
new_names = ["Instructor"]
new_names.extend(math_courses)
new_names.extend(ds_courses)
new_names
course_preferences.columns = new_names
course_preferences

,Instructor,Math 201,Math 202,Math 302,Math 304,Math 322,Math 365,Math 451,Math 260,Math 350,...,Math 10,Math 20,Math 50,Math 291,Math 297,DS 140,DS 150,DS 250,DS 710,DS 740
1,AbraBrisbin,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,5.0
2,JulianAntolinCamarena,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.0,3.0,0.0,5.0,5.0
3,ChrisAhrendt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ColleenDuffy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,MohammadAziz,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0
6,AllisonBeemer,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,3.0,0.0
7,AihamHassan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,ChloeLewis,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,AbhinavChand,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,3.0,5.0,5.0
10,VickiWhitledge,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
#course_preferences.iloc[:,20:30]

In [10]:
with pd.ExcelWriter(output_file_name) as writer:  
    course_preferences.to_excel(writer, sheet_name='Preferences', index=False)

## Time preferences

In [11]:
#detailed_header[69]

In [12]:
#preferences.iloc[:,66]

In [13]:
def translate_time(n):
    # If n <= 12, returns n (n is in the morning)
    # If n >= 13, returns n-12 (n is in the afternoon)
    if n <= 12:
        return n
    else:
        return n-12
    
def hour_to_24(n):
    # If n >= 8 returns n (n is in the morning)
    # If n <= 7, returns n+12 (n is in the afternoon)
    if n >= 8:
        return n
    else:
        return n+12

### List the Qualtrics indices associated with time options.  Make a dataframe that I can compare with meeting_patterns to determine overlaps

In [14]:
# ID possible time options.  Here, we assume that the Qualtrics form uses 1-hour increments and 
# doesn't skip any times between starting at 8 am and ending at 7 pm each day.
time_options = []
day_list = []
day_options = {1:"M", 2:"T", 3:"W", 4:"R", 5:"F"}
start_time_list = []
current_time = 8
current_day = 1
index = 2
while current_day <= 5:
    while current_time <= 18:
        label = f'{index}'#': {current_time%12}:00-{(current_time+1)%12}:00'
        time_options.append(label)
        day_list.append(day_options[current_day])
        #start_time_list.append(translate_time(current_time))
        start_time_list.append(current_time)
        index += 1
        current_time += 1
    # Change the day
    current_day += 1
    index += 1 # this double-augments the index, because Qualtrics counts the name of the day of the week
    current_time = 8

time_connector = pd.DataFrame({'Pref_by_day_index':time_options, 'Day':day_list, 'Hour':start_time_list})
#time_options
time_connector

,Pref_by_day_index,Day,Hour
0,2,M,8
1,3,M,9
2,4,M,10
3,5,M,11
4,6,M,12
5,7,M,13
6,8,M,14
7,9,M,15
8,10,M,16
9,11,M,17


In [15]:
time_prefs_by_day = pd.DataFrame(index = range(preferences.shape[0]), columns = time_options)
time_prefs_by_day = time_prefs_by_day.infer_objects()
time_prefs_by_day = time_prefs_by_day.fillna(3) # default value:  3 for "OK"

In [16]:
# Dictionary relating how people feel about a time with what score it gets in the objective function
time_opinions = {"Prefer":5, "Dislike":1, "Impossible":0} # No need to include "OK" because it's the default

In [17]:
# Add instructors' names as row labels, so we can use .loc on them below
time_prefs_by_day.set_index(preferences["Prof_name"], inplace=True)
time_prefs_by_day

,2,3,4,5,6,7,8,9,10,11,...,51,52,53,54,55,56,57,58,59,60
Prof_name,,,,,,,,,,,,,,,,,,,,,
AbraBrisbin,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
JulianAntolinCamarena,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
ChrisAhrendt,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
ColleenDuffy,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
MohammadAziz,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
AllisonBeemer,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
AihamHassan,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
ChloeLewis,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
AbhinavChand,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0


In [18]:
for (key, value) in time_opinions.items():
    # Identify the column of preferences that contains this type of time preference
    time_col = []
    for col in range(detailed_header.size):
        pattern = "Click, drag, and select .* - " + key
        found_time_pref = re.findall(pattern, detailed_header.iloc[col])
        if found_time_pref:
            time_col.append(col)

    #print(time_col)
    #print(value)

    for person in range(preferences.shape[0]):
        person_name = preferences.Prof_name.iloc[person]
        enjoy_times = preferences.iloc[person, time_col[0]]
        if isinstance(enjoy_times, str): # NA's will be floats
            enjoy_times = enjoy_times.split(",")
            for entry in enjoy_times:
                time_index = entry.split(":")[0]
                #time_prefs_by_day[time_index].iloc[person] = value # This bit gave the FutureWarning
                time_prefs_by_day.loc[person_name,time_index] = value # This method fixes the FutureWarning      

print(time_prefs_by_day)

                         2    3    4    5    6    7    8    9   10   11  ...  \
Prof_name                                                                ...   
AbraBrisbin            1.0  1.0  3.0  3.0  3.0  3.0  3.0  3.0  3.0  3.0  ...   
JulianAntolinCamarena  5.0  5.0  5.0  5.0  5.0  5.0  5.0  5.0  5.0  3.0  ...   
ChrisAhrendt           5.0  5.0  5.0  5.0  0.0  3.0  1.0  0.0  0.0  0.0  ...   
ColleenDuffy           1.0  3.0  3.0  3.0  1.0  3.0  3.0  1.0  1.0  0.0  ...   
MohammadAziz           0.0  1.0  3.0  5.0  5.0  5.0  5.0  0.0  0.0  0.0  ...   
AllisonBeemer          1.0  5.0  5.0  3.0  5.0  3.0  3.0  1.0  0.0  0.0  ...   
AihamHassan            3.0  3.0  3.0  3.0  3.0  3.0  3.0  3.0  3.0  3.0  ...   
ChloeLewis             3.0  3.0  3.0  3.0  3.0  3.0  3.0  3.0  3.0  3.0  ...   
AbhinavChand           1.0  3.0  3.0  3.0  3.0  3.0  3.0  3.0  3.0  3.0  ...   
VickiWhitledge         0.0  5.0  3.0  3.0  5.0  3.0  3.0  1.0  0.0  0.0  ...   
RobertBerg             3.0  3.0  3.0  3.

## Relate time preferences-by-hour (from Qualtrics) to meeting patterns


For every meeting pattern, take the minimum score for a time that overlaps that pattern.

This code would need major changes if we change to half-hour increments in Qualtrics.

In [20]:
time_prefs_all = pd.DataFrame(index = range(preferences.shape[0]), columns = meeting_patterns["Time_Index"].unique())
time_prefs_all.set_index(preferences["Prof_name"], inplace=True)
time_prefs_all[0] = 3 # "No meeting pattern"  is "OK" for everyone
time_prefs_all
#time_prefs_all.loc[:,50:60]

,0,1,2,3,4,5,6,7,8,9,...,132,133,134,135,136,137,138,139,140,141
Prof_name,,,,,,,,,,,,,,,,,,,,,
AbraBrisbin,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
JulianAntolinCamarena,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ChrisAhrendt,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ColleenDuffy,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MohammadAziz,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AllisonBeemer,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AihamHassan,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ChloeLewis,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AbhinavChand,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
meeting_patterns["Start_hour"] = pd.to_datetime(meeting_patterns["Start_Time"], format='%H:%M:%S').dt.hour
meeting_patterns["End_hour"] = pd.to_datetime(meeting_patterns["End_Time"], format='%H:%M:%S').dt.hour

meeting_patterns#['Days'][time_index]

,Time_Index,Days,Start_Time,End_Time,Time_Group,Notes,Start_hour,End_hour
0,0,NaN,NaN,NaN,M,No scheduled time,NaN,NaN
1,1,MWF,08:00:00,08:50:00,E,NaN,8.0,8.0
2,2,MWF,09:00:00,09:50:00,E,NaN,9.0,9.0
3,3,MWF,10:00:00,10:50:00,E,NaN,10.0,10.0
4,4,MWF,11:00:00,11:50:00,E,NaN,11.0,11.0
...,...,...,...,...,...,...,...,...
153,137,MTWR,12:00:00,12:50:00,P,NaN,12.0,12.0
154,138,MTWR,01:00:00,01:50:00,P,NaN,1.0,1.0
155,139,MTWR,02:00:00,02:50:00,P,NaN,2.0,2.0
156,140,MTWR,03:00:00,03:50:00,P,NaN,3.0,3.0


In [22]:
meeting_patterns["Start_hour"] = meeting_patterns["Start_hour"].apply(hour_to_24)
meeting_patterns["End_hour"] = meeting_patterns["End_hour"].apply(hour_to_24)

# Hack to fix the fact that 8 pm is a valid ending time
afternoon = meeting_patterns["Start_hour"] > 12
end_at_8 = meeting_patterns["End_hour"] == 8
meeting_patterns.loc[afternoon & end_at_8, "End_hour"] = 20

meeting_patterns.iloc[55:75,:]

,Time_Index,Days,Start_Time,End_Time,Time_Group,Notes,Start_hour,End_hour
55,55,MWF,12:00:00,12:50:00,L,NaN,12.0,12.0
56,55,T,12:00:00,01:50:00,L,NaN,12.0,13.0
57,56,MWF,01:00:00,01:50:00,L,NaN,13.0,13.0
58,56,T,01:00:00,02:50:00,L,NaN,13.0,14.0
59,57,MWF,02:00:00,02:50:00,L,NaN,14.0,14.0
60,57,T,02:00:00,02:50:00,L,NaN,14.0,14.0
61,58,MWF,03:00:00,03:50:00,L,NaN,15.0,15.0
62,58,T,03:00:00,04:50:00,L,NaN,15.0,16.0
63,59,MWF,08:00:00,08:50:00,L,NaN,8.0,8.0
64,59,R,08:00:00,09:50:00,L,NaN,8.0,9.0


In [23]:
for time_index in meeting_patterns.Time_Index[1:]:
    meeting_rows = (meeting_patterns.Time_Index == time_index) # row(s) of meeting_patterns that match this time_index

    meeting_subset = meeting_patterns[meeting_rows]
    is_overlap = False
    #print(meeting_subset)
    for row in range(meeting_subset.shape[0]):
        # This bit produces True for all entries in time_connector that are on a day 
        # that matches the current time_index in meeting_patterns
        day_result = False
        for one_day in meeting_subset['Days'].iloc[row]:
            day_result = day_result | (time_connector['Day'] == one_day)
    
        end_overlap = time_connector['Hour'] <= meeting_subset['End_hour'].iloc[row]
        start_overlap = time_connector['Hour'] >= meeting_subset['Start_hour'].iloc[row]
        # Find times that overlap on days that overlap.
        # The `is_overlap |` deals with the fact that some meeting patterns have different times on different days.
        is_overlap = is_overlap | (end_overlap & start_overlap & day_result)

    # Find the Qualtrics time indices that overlap with this time_index from meeting_patterns
    matching_times = time_connector.loc[is_overlap, "Pref_by_day_index"]
    #print(matching_times)

    # Take min over columns (times), for each row (person)
    time_prefs_all.loc[:,time_index] = time_prefs_by_day.loc[:, matching_times].min(axis = 1) 



In [24]:
time_prefs_all.iloc[:,10:17]
time_prefs_all.iloc[:,65:75]

,65,66,67,68,69,70,71,72,73,74
Prof_name,,,,,,,,,,
AbraBrisbin,1.0,0.0,3.0,3.0,3.0,3.0,1.0,1.0,3.0,3.0
JulianAntolinCamarena,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
ChrisAhrendt,5.0,5.0,0.0,1.0,0.0,0.0,5.0,5.0,5.0,0.0
ColleenDuffy,1.0,3.0,1.0,0.0,0.0,0.0,1.0,3.0,3.0,1.0
MohammadAziz,0.0,1.0,5.0,5.0,0.0,0.0,0.0,1.0,3.0,5.0
AllisonBeemer,1.0,3.0,3.0,3.0,1.0,0.0,1.0,5.0,3.0,3.0
AihamHassan,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
ChloeLewis,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
AbhinavChand,1.0,3.0,3.0,3.0,3.0,3.0,1.0,3.0,3.0,3.0


### Deal with different meeting patterns of Math 246

Add 0.5 if it's a "preferred" length, subtract 0.5 if it's a "disliked" length.  Do not change 0's.
Using 0.5 so that "disliked time" + "disliked length" doesn't result in a preference of 0, which would look like "impossible."

In [25]:
#preferences.columns.get_loc("50 minutes, 4 days per week")
#[col for col in range(detailed_header.size) if detailed_header.iloc[col].find("Math ") > -1]

In [26]:
# def add_length_pref(column, **kwargs):
#     blah = kwargs.get('to_add', 17)
#     print(blah)
#     to_add = [1,2,3]
#     column = column.iloc[:,0].to_list() # Convert it from a 1-column DF to a Series, then to a list
#     column = [c + new_val if c != 0 else 0
#              for c, new_val in zip(column, to_add)]

#     return column 

In [27]:
# Note that if we were calling this function without apply(), like this:
#     my_test = pd.DataFrame({"col1":[5,0,7]})
#     add_length_pref(my_test, to_add=[1,2,3])
# then we would need column.iloc[:,0].to_list() instead of just column.to_list()

def add_length_pref(column, to_add):
    #to_add = hypothetical
    column = column.to_list() # Convert it to a list.
                              # If we were calling it as from a 1-column DF to a Series, then .iloc[:,0]
    column = [c + new_val if c != 0 else 0
             for c, new_val in zip(column, to_add)]
    return column 

In [28]:
time_prefs_all

,0,1,2,3,4,5,6,7,8,9,...,132,133,134,135,136,137,138,139,140,141
Prof_name,,,,,,,,,,,,,,,,,,,,,
AbraBrisbin,3,1.0,1.0,1.0,3.0,1.0,3.0,3.0,3.0,1.0,...,3.0,3.0,3.0,1.0,1.0,3.0,3.0,3.0,3.0,3.0
JulianAntolinCamarena,3,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
ChrisAhrendt,3,5.0,5.0,5.0,5.0,0.0,3.0,1.0,0.0,5.0,...,1.0,0.0,0.0,5.0,5.0,0.0,3.0,1.0,0.0,0.0
ColleenDuffy,3,1.0,3.0,3.0,3.0,1.0,3.0,3.0,1.0,1.0,...,0.0,0.0,0.0,1.0,3.0,1.0,3.0,0.0,0.0,0.0
MohammadAziz,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,1.0,5.0,5.0,5.0,0.0,0.0
AllisonBeemer,3,1.0,5.0,5.0,3.0,5.0,3.0,3.0,1.0,1.0,...,3.0,1.0,0.0,1.0,3.0,3.0,3.0,3.0,1.0,0.0
AihamHassan,3,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
ChloeLewis,3,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
AbhinavChand,3,1.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,1.0,...,3.0,3.0,3.0,1.0,3.0,3.0,3.0,3.0,3.0,3.0


If the cell below gives "ValueError: Columns must be same length as key," your Time Options might have two rows with the same number that aren't actually part of the same meeting pattern.

In [29]:
length_options = {"50 minutes, 4 days per week":"H", "75 minutes, 3 days per week":"J", "110 minutes, 2 days per week":"K"}
for (selected_length,time_gp) in length_options.items():
    # Identify the column of preferences that contains this type of time preference
    length_col = [col for col in range(detailed_header.size) if detailed_header.iloc[col].find(selected_length) > -1]
    #print(selected_length)
    length_option_pref = preferences.iloc[:,length_col[0]] # Series with 1 entry per instructor
    #print(length_option_pref)
    
    # Create the Series to add to existing time prefs.
    # Use length_option_pref.index (may not be 0 to n-1) to make it compatible with .loc
    length_option_pref_numeric = pd.Series(float(0), index = length_option_pref.index) # Neutral and NaN are represented by 0's
    
    length_option_pref_numeric.loc[length_option_pref == "Prefer"] = 0.5
    length_option_pref_numeric.loc[length_option_pref == "Dislike"] = -0.5
    #print(length_option_pref_numeric)
    times_to_update = meeting_patterns.loc[meeting_patterns["Time_Group"] == time_gp, "Time_Index"].to_list()
    #print(times_to_update)
    #print(time_prefs_all[times_to_update])
    #print("here's what we want to change it to:")
    #print(time_prefs_all[times_to_update].apply(add_length_pref, axis=0, to_add=length_option_pref_numeric))

    #time_prefs_all = time_prefs_all.infer_objects()
    time_prefs_all[times_to_update] = time_prefs_all[times_to_update].apply(add_length_pref, axis=0, to_add=length_option_pref_numeric)
    #print(time_prefs_all[times_to_update])


In [30]:
#time_prefs_all.infer_objects().dtypes
#length_option_pref_numeric = pd.Series(0, index = length_option_pref.index)
#length_option_pref_numeric.dtypes

In [31]:
#times_to_update

In [32]:
#time_prefs_all[times_to_update].apply(add_length_pref, axis=0, to_add=length_option_pref_numeric)

In [33]:
#time_prefs_all[times_to_update]

In [34]:
#time_prefs_all.iloc[:,10:17]
#time_prefs_all.iloc[:,73:80]

### Append instructor names and output results to file

In [35]:
#time_prefs_all.insert(0, "Prof_name", preferences["FullName"].to_list())
#time_prefs_all

In [36]:
with pd.ExcelWriter(output_file_name, mode="a", engine="openpyxl") as writer:  
    time_prefs_all.to_excel(writer, sheet_name='Time_preferences', index=True)

In [37]:
meeting_patterns.iloc[:9, :]

,Time_Index,Days,Start_Time,End_Time,Time_Group,Notes,Start_hour,End_hour
0,0,NaN,NaN,NaN,M,No scheduled time,NaN,NaN
1,1,MWF,08:00:00,08:50:00,E,NaN,8.0,8.0
2,2,MWF,09:00:00,09:50:00,E,NaN,9.0,9.0
3,3,MWF,10:00:00,10:50:00,E,NaN,10.0,10.0
4,4,MWF,11:00:00,11:50:00,E,NaN,11.0,11.0
5,5,MWF,12:00:00,12:50:00,E,NaN,12.0,12.0
6,6,MWF,01:00:00,01:50:00,E,NaN,13.0,13.0
7,7,MWF,02:00:00,02:50:00,E,NaN,14.0,14.0
8,8,MWF,03:00:00,03:50:00,E,NaN,15.0,15.0


## Back-to-back courses

We assume that the Qualtrics questions are listed in the order two_same, two_diff, three_b2b.

In [38]:
found_b2b = [col for col in range(detailed_header.size) if detailed_header.iloc[col].find("back-to-back") > -1]
b2b_prefs = preferences.iloc[:,found_b2b]
b2b_prefs.columns = ["Two_same_b2b", "Two_diff_b2b", "Three_b2b"]
b2b_prefs.insert(0, "Prof_name", preferences["Prof_name"].to_list())
b2b_prefs = b2b_prefs.infer_objects()
b2b_prefs = b2b_prefs.fillna(0)
b2b_prefs

,Prof_name,Two_same_b2b,Two_diff_b2b,Three_b2b
1,AbraBrisbin,4.0,4.0,-3.0
2,JulianAntolinCamarena,4.0,0.0,-2.0
3,ChrisAhrendt,4.0,4.0,3.0
4,ColleenDuffy,4.0,3.0,-2.0
5,MohammadAziz,5.0,0.0,0.0
6,AllisonBeemer,3.0,2.0,-1.0
7,AihamHassan,0.0,0.0,0.0
8,ChloeLewis,5.0,0.0,0.0
9,AbhinavChand,5.0,4.0,0.0
10,VickiWhitledge,0.0,-2.0,-3.0


## Preps

In [39]:
found_preps = [col for col in range(detailed_header.size) if detailed_header.iloc[col].find("preps") > -1]
prep_prefs = preferences.iloc[:,found_preps]
prep_prefs.columns = ["One", "Two", "Three", "Four"]
prep_prefs.insert(0, "Prof_name", preferences["Prof_name"].to_list())
prep_prefs = prep_prefs.infer_objects()
prep_prefs = prep_prefs.fillna(0)
prep_prefs

,Prof_name,One,Two,Three,Four
1,AbraBrisbin,5.0,5.0,0.0,0.0
2,JulianAntolinCamarena,5.0,5.0,3.0,0.0
3,ChrisAhrendt,0.0,4.0,0.0,-3.0
4,ColleenDuffy,3.0,5.0,2.0,-5.0
5,MohammadAziz,0.0,5.0,0.0,-5.0
6,AllisonBeemer,5.0,4.0,-3.0,-5.0
7,AihamHassan,0.0,0.0,0.0,0.0
8,ChloeLewis,5.0,5.0,0.0,-5.0
9,AbhinavChand,5.0,5.0,-1.0,-5.0
10,VickiWhitledge,5.0,2.0,-3.0,-5.0


In [40]:
with pd.ExcelWriter(output_file_name, mode="a", engine="openpyxl") as writer:  
    b2b_prefs.to_excel(writer, sheet_name='Back_to_back', index=False)
    prep_prefs.to_excel(writer, sheet_name='Preps', index=False)